(https://elitedatascience.com/keras-tutorial-deep-learning-in-python)

(https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/)

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

In [3]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11190272/11490434 [============================>.] - ETA: 0s

In [4]:
# reshape to be [samples][depth][width][height]
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols).astype('float32')
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols).astype('float32')
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1).astype('float32')
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1).astype('float32')
    input_shape = (img_rows, img_cols, 1)

In [5]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
# normalize inputs from 0-255 to 0-1
X_train /= 255
X_test /= 255

In [7]:
print(y_train[:10])

[5 0 4 1 9 2 1 3 1 4]


In [8]:
import numpy as np
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [9]:
num_classes = 10

In [10]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [11]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5, 5), strides=(1,1),
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [12]:
def medium_model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    return model

In [13]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Conv2D(filters=30, kernel_size=(5, 5), strides=(1,1),
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [19]:
# build the model
#model = baseline_model()
#model = medium_model()
model = larger_model()

In [20]:
batch_size = 128
epochs = 12

# Fit the model
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=epochs,
          batch_size=batch_size,
          verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 8s - loss: 0.3257 - acc: 0.8957 - val_loss: 0.0710 - val_acc: 0.9779
Epoch 2/12
60000/60000 [==============================] - 7s - loss: 0.0849 - acc: 0.9733 - val_loss: 0.0463 - val_acc: 0.9851
Epoch 3/12
60000/60000 [==============================] - 7s - loss: 0.0620 - acc: 0.9808 - val_loss: 0.0327 - val_acc: 0.9890
Epoch 4/12
60000/60000 [==============================] - 7s - loss: 0.0501 - acc: 0.9839 - val_loss: 0.0380 - val_acc: 0.9876
Epoch 5/12
60000/60000 [==============================] - 7s - loss: 0.0448 - acc: 0.9857 - val_loss: 0.0322 - val_acc: 0.9892
Epoch 6/12
60000/60000 [==============================] - 7s - loss: 0.0400 - acc: 0.9873 - val_loss: 0.0291 - val_acc: 0.9914
Epoch 7/12
60000/60000 [==============================] - 7s - loss: 0.0333 - acc: 0.9894 - val_loss: 0.0324 - val_acc: 0.9892
Epoch 8/12
60000/60000 [==============================] - 7s 

In [21]:
# Final evaluation of the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.024958568213
Test accuracy: 0.9919


In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 24, 24, 30)        780       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 30)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 15)        4065      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 15)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 15)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 375)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               48128     
__________